In [1]:
import pandas as pd, numpy as np
from datetime import datetime
pd.set_option('max_colwidth',200)
# import yfinance as yf
import tushare as ts
ts.set_token('2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67')
import time, urllib
import glob
import os

In [2]:
def get_data(code,start,end):
    df = ts.pro_bar(ts_code=code, adj='qfq', start_date=start, end_date=end)
    return df

#获取当前交易日最新的股票代码和简称
def get_code():
    codes = pro.stock_basic(list_status='L').ts_code.values
    return codes


def download_tushare_stocks_data(start, end, ch_db_path, ticker_list):
    count = 1
    for ticker in ticker_list:
        if count%200==0:
            print("=======================Sleeping======================")
            time.sleep(60)
        else:
            if not os.path.exists(ch_db_path+ticker+".csv"):
                print("{} is new, start downloading now...".format(ticker))
                try:
                    data = get_data(ticker, start, end)
    #                 print(data)
                    data['trade_date'] = data['trade_date'].astype(str)
                    data['trade_date'] = data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                    data.sort_values("trade_date", inplace = True)
                    data.to_csv(ch_db_path+ticker+".csv", index = False)
                    print("{} data file created: {}".format(ticker, end))
                except Exception as e:
                    print(e)

            else:
                print("Already have data csv for {}".format(ticker))
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")   
                hist_data['trade_date'] = hist_data['trade_date'].astype(str)
                hist_data['trade_date'] = hist_data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                hist_data.to_csv(ch_db_path+ticker+".csv", index = False)
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")
                try:
                    hist_data_last_date = hist_data['trade_date'].values[-1]        
                    if today > hist_data_last_date:
                        print("Needs to update, start updating new data for {} now...".format(ticker))
                        update_start = hist_data_last_date
                        update_end = today
                        with eventlet.Timeout(60,False):
                            try:
                                new_data = get_data(ticker, update_start, update_end)
                                new_data['trade_date'] = new_data['trade_date'].astype(str)
                                new_data['trade_date'] = new_data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:])
                                new_data.to_csv(ch_db_path+ticker+".csv", mode='a', header=False, index = False)
                                updated_duplicated_df = pd.read_csv(ch_db_path+ticker+".csv")
                                updated_df = updated_duplicated_df.drop_duplicates("trade_date")
                                updated_df.sort_values("trade_date", inplace = True)
                                updated_df.to_csv(ch_db_path+ticker+".csv", index = False)
                                print("New data updated till today for {}!".format(ticker))
                            except Exception as e:
                                print(e)
            #             print("Timed Out: Update Failed!")
                    else:
                        print("There's no new data to update for {}.".format(ticker))
                except Exception as e:
                    print(e)

        #     print("Data Download/Update for {} is Finished.".format(ticker))
            print("=======================Executed: {}=======================".format(count))
        count+=1
    print("【Updated Finished for today!】")


#设置token
token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
codes = get_code()
print("Public Company Numbers by today: ", len(codes))

today = str(datetime.now().date())
start = '20100101'
end = today
ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
ticker_list = codes

Public Company Numbers by today:  3758


In [3]:
import eventlet
eventlet.monkey_patch()

download_tushare_stocks_data(start, end, ch_db_path, ticker_list)

Already have data csv for 000001.SZ
Needs to update, start updating new data for 000001.SZ now...


/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)
/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


New data updated till today for 000001.SZ!
=======================Executed: 1=======================
Already have data csv for 000002.SZ
Needs to update, start updating new data for 000002.SZ now...
New data updated till today for 000002.SZ!
=======================Executed: 2=======================
Already have data csv for 000004.SZ
Needs to update, start updating new data for 000004.SZ now...
New data updated till today for 000004.SZ!
=======================Executed: 3=======================
Already have data csv for 000005.SZ
Needs to update, start updating new data for 000005.SZ now...
New data updated till today for 000005.SZ!
=======================Executed: 4=======================
Already have data csv for 000006.SZ
Needs to update, start updating new data for 000006.SZ now...
New data updated till today for 000006.SZ!
=======================Executed: 5=======================
Already have data csv for 000007.SZ
Needs to update, start updating new data for 000007.SZ now...
New d

New data updated till today for 000059.SZ!
=======================Executed: 43=======================
Already have data csv for 000060.SZ
Needs to update, start updating new data for 000060.SZ now...
New data updated till today for 000060.SZ!
=======================Executed: 44=======================
Already have data csv for 000061.SZ
Needs to update, start updating new data for 000061.SZ now...
New data updated till today for 000061.SZ!
=======================Executed: 45=======================
Already have data csv for 000062.SZ
Needs to update, start updating new data for 000062.SZ now...
New data updated till today for 000062.SZ!
=======================Executed: 46=======================
Already have data csv for 000063.SZ
Needs to update, start updating new data for 000063.SZ now...
New data updated till today for 000063.SZ!
=======================Executed: 47=======================
Already have data csv for 000065.SZ
Needs to update, start updating new data for 000065.SZ now...


New data updated till today for 000416.SZ!
=======================Executed: 84=======================
Already have data csv for 000417.SZ
Needs to update, start updating new data for 000417.SZ now...
New data updated till today for 000417.SZ!
=======================Executed: 85=======================
Already have data csv for 000419.SZ
Needs to update, start updating new data for 000419.SZ now...
New data updated till today for 000419.SZ!
=======================Executed: 86=======================
Already have data csv for 000420.SZ
Needs to update, start updating new data for 000420.SZ now...
New data updated till today for 000420.SZ!
=======================Executed: 87=======================
Already have data csv for 000421.SZ
Needs to update, start updating new data for 000421.SZ now...
New data updated till today for 000421.SZ!
=======================Executed: 88=======================
Already have data csv for 000422.SZ
Needs to update, start updating new data for 000422.SZ now...


...

New data updated till today for 603817.SH!
=======================Executed: 3586=======================
Already have data csv for 603818.SH
Needs to update, start updating new data for 603818.SH now...
New data updated till today for 603818.SH!
=======================Executed: 3587=======================
Already have data csv for 603819.SH
Needs to update, start updating new data for 603819.SH now...
New data updated till today for 603819.SH!
=======================Executed: 3588=======================
Already have data csv for 603822.SH
Needs to update, start updating new data for 603822.SH now...
New data updated till today for 603822.SH!
=======================Executed: 3589=======================
Already have data csv for 603823.SH
Needs to update, start updating new data for 603823.SH now...
New data updated till today for 603823.SH!
=======================Executed: 3590=======================
Already have data csv for 603825.SH
Needs to update, start updating new data for 603825.

New data updated till today for 603897.SH!
=======================Executed: 3628=======================
Already have data csv for 603898.SH
Needs to update, start updating new data for 603898.SH now...
New data updated till today for 603898.SH!
=======================Executed: 3629=======================
Already have data csv for 603899.SH
Needs to update, start updating new data for 603899.SH now...
New data updated till today for 603899.SH!
=======================Executed: 3630=======================
Already have data csv for 603900.SH
Needs to update, start updating new data for 603900.SH now...
New data updated till today for 603900.SH!
=======================Executed: 3631=======================
Already have data csv for 603901.SH
Needs to update, start updating new data for 603901.SH now...
New data updated till today for 603901.SH!
=======================Executed: 3632=======================
Already have data csv for 603903.SH
Needs to update, start updating new data for 603903.

New data updated till today for 603980.SH!
=======================Executed: 3669=======================
Already have data csv for 603982.SH
Needs to update, start updating new data for 603982.SH now...
New data updated till today for 603982.SH!
=======================Executed: 3670=======================
Already have data csv for 603983.SH
Needs to update, start updating new data for 603983.SH now...
New data updated till today for 603983.SH!
=======================Executed: 3671=======================
Already have data csv for 603985.SH
Needs to update, start updating new data for 603985.SH now...
New data updated till today for 603985.SH!
=======================Executed: 3672=======================
Already have data csv for 603986.SH
Needs to update, start updating new data for 603986.SH now...
New data updated till today for 603986.SH!
=======================Executed: 3673=======================
Already have data csv for 603987.SH
Needs to update, start updating new data for 603987.

New data updated till today for 688036.SH!
=======================Executed: 3710=======================
Already have data csv for 688037.SH
Needs to update, start updating new data for 688037.SH now...
New data updated till today for 688037.SH!
=======================Executed: 3711=======================
Already have data csv for 688039.SH
Needs to update, start updating new data for 688039.SH now...
New data updated till today for 688039.SH!
=======================Executed: 3712=======================
Already have data csv for 688058.SH
Needs to update, start updating new data for 688058.SH now...
New data updated till today for 688058.SH!
=======================Executed: 3713=======================
Already have data csv for 688066.SH
Needs to update, start updating new data for 688066.SH now...
New data updated till today for 688066.SH!
=======================Executed: 3714=======================
Already have data csv for 688068.SH
Needs to update, start updating new data for 688068.

New data updated till today for 688363.SH!
=======================Executed: 3752=======================
Already have data csv for 688366.SH
Needs to update, start updating new data for 688366.SH now...
New data updated till today for 688366.SH!
=======================Executed: 3753=======================
Already have data csv for 688368.SH
Needs to update, start updating new data for 688368.SH now...
New data updated till today for 688368.SH!
=======================Executed: 3754=======================
Already have data csv for 688369.SH
Needs to update, start updating new data for 688369.SH now...
New data updated till today for 688369.SH!
=======================Executed: 3755=======================
Already have data csv for 688388.SH
Needs to update, start updating new data for 688388.SH now...
New data updated till today for 688388.SH!
=======================Executed: 3756=======================
Already have data csv for 688389.SH
Needs to update, start updating new data for 688389.

In [ ]:
tick